<a href="https://colab.research.google.com/github/markNZed/GPT-NeoX-Colab/blob/main/notebooks/neox_python3.8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

It is not obvious how to downgrade the Colab Python from 3.10 to 3.8 for GPT-NeoX There are some ideas at https://stackoverflow.com/questions/60775160/install-python-3-8-kernel-in-google-colaboratory

In [ ]:
import sys
print(sys.version)

3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]


In [ ]:
!apt-get update -y
!apt-get install python3.8 python3.8-dev
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!python --version
!apt-get install python3.8-distutils
!wget https://bootstrap.pypa.io/get-pip.py
!python3.8 get-pip.py
!python3.8 -m pip --version
!update-alternatives --install /usr/bin/pip3 pip3 /usr/bin/python3.8 1

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,071 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
G

In [ ]:
!git clone https://github.com/EleutherAI/gpt-neox.git
%cd /content/gpt-neox

fatal: destination path 'gpt-neox' already exists and is not an empty directory.
/content/gpt-neox


In [ ]:
!ls

CITATION.cff		      docker-compose.yml  images	 prepare_data.py  tools
configs			      Dockerfile	  LICENSE	 README.md	  train.py
CONTRIBUTING.md		      eval.py		  MANIFEST.in	 README-MUP.md
deepy.py		      eval_tasks	  megatron	 requirements
docker-compose-dockerhub.yml  generate.py	  post-training  tests


In [ ]:
!pip install -r requirements/requirements.txt

In [ ]:
%pip show deepspeed


In [ ]:
import requests
from pathlib import Path

# Download Shakespeare data
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
Path("shakespeare.txt").write_text(response.text)

# Split text into chunks
with open("shakespeare.txt") as f:
    text = f.read()

chunk_size = 256
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Save as JSONL format
with open("shakespeare.jsonl", "w") as f:
    for chunk in chunks:
        f.write(f'{{"text": "{chunk}"}}\n')

In [ ]:
from tokenizers import ByteLevelBPETokenizer

import os

# Create the directory if it doesn't exist
os.makedirs("tokenizer", exist_ok=True)

tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=["shakespeare.txt"], vocab_size=5000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])
tokenizer.save_model("tokenizer")

['tokenizer/vocab.json', 'tokenizer/merges.txt']

In [ ]:
import yaml

# Define the configuration data as a dictionary
config_data = {
    "vocab-file": "shakespeare_tokenizer.json",
    "hidden-size": 384,
    "num-attention-heads": 6,
    "num-layers": 6,
    "max-position-embeddings": 256,
    "micro-batch-size": 32,
    "seq-length": 256,
    "train-iters": 5000,
    "lr-decay-iters": 5000,
    "lr": 6e-4,
    "min-lr": 6e-5,
    "warmup": 0.01,
    "adam-beta1": 0.9,
    "adam-beta2": 0.95,
    "weight-decay": 0.1,
    "clip-grad": 1.0,
    "fp16": True
}

# Write the dictionary to a YAML file
with open("shakespeare_config.yml", "w") as f:
    yaml.dump(config_data, f, default_flow_style=False)


In [ ]:
import deepspeed

In [ ]:
%run deepy.py train.py -d configs shakespeare_config.yml